In [2]:
import findspark
findspark.init()
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import col,lit
import pydeequ
import pyspark
from pyspark.sql.types import *
from pydeequ import Check,CheckLevel
from pydeequ.checks import *
from pydeequ.verification import *
from pydeequ.analyzers import *
from pyspark.sql import SparkSession
from configparser import ConfigParser
import sys


In [3]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.jars", "C:\Program Files (x86)\PostgreSQL\pgJDBC\postgresql-42.2.18.jar") \
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)\
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)\
    .getOrCreate()

In [9]:
Parser = ConfigParser()
configFile = r"C:\Users\Shivam\Desktop\training\godaddy\godaddy\config\Config.json"
Parser.read(configFile)
Env = "dev"
#SourceFileName = "merchants.csv"

##Set Job Parameters
Analyzer_S3_Path = Parser.get(Env, 'Analyzer_S3_Path')
Clean_S3_Path = Parser.get(Env, 'Clean_S3_Path')
Format = Parser.get(Env, 'Format')
print("Analyzer_S3_Path = " + Analyzer_S3_Path)
print("Clean_S3_Path = " + Clean_S3_Path)
print("Format = " + Format)
#print("SourceFile = " + S3_Path )

Analyzer_S3_Path = C:\\Users\\Shivam\\Desktop\\training\\godaddy\\godaddy\\analyzer_files\\merchant_analyzer
Clean_S3_Path = C:\Users\Shivam\Desktop\training\godaddy\godaddy\data\\merchants.csv
Format = csv


In [10]:
modified_attr = Parser.get(Env,'Modified_Attributes')
new_checks = Parser.get(Env,'New_Checks')

In [11]:
print(modified_attr)
print("\n",new_checks)

id:merchants_id, updated_at:modified_at, telephone:contact_no

 Completeness('state'), Completeness('modified_at')


In [12]:
df = spark.read.format(Format).load(Clean_S3_Path,header=True,inferSchema=True).select('id','state','updated_at','telephone')

In [13]:
analyzer_df = spark.read.format(Format).load(Analyzer_S3_Path,header=True,inferSchema=True)
filtered  = analyzer_df.filter(analyzer_df.source_table == 'merchants')

In [14]:
filtered.show()

+-------+---------+------------+--------+--------------------+------------+
| entity| instance|        name|   value|       ingestion_tsp|source_table|
+-------+---------+------------+--------+--------------------+------------+
| Column|       id|     Maximum|174577.0|2020-12-21 16:55:...|   merchants|
| Column|telephone|Completeness|   0.999|2020-12-21 16:55:...|   merchants|
| Column|       id|Distinctness|     1.0|2020-12-21 16:55:...|   merchants|
| Column|       id|Completeness|     1.0|2020-12-21 16:55:...|   merchants|
| Column|       id|     Minimum|   469.0|2020-12-21 16:55:...|   merchants|
|Dataset|        *|        Size|  1000.0|2020-12-21 16:55:...|   merchants|
+-------+---------+------------+--------+--------------------+------------+



In [15]:
instance = filtered.select("instance").rdd.map(lambda r : r[0]).collect()
values = filtered.select("value").rdd.map(lambda r : r[0]).collect()
names = filtered.select("name").rdd.map(lambda r : r[0]).collect()

In [16]:
print(instance)
print(values)
print(names)

['id', 'telephone', 'id', 'id', 'id', '*']
[174577.0, 0.999, 1.0, 1.0, 469.0, 1000.0]
['Maximum', 'Completeness', 'Distinctness', 'Completeness', 'Minimum', 'Size']


In [17]:
print(modified_attr)
print("\n",new_checks)

id:merchants_id, updated_at:modified_at, telephone:contact_no

 Completeness('state'), Completeness('modified_at')


In [18]:
checkers = {'Completeness':'isComplete()','Distinctness':'isUnique()','Maximum':'hasMax()','Minimum':'hasMin()','Size':'hasSize()'}

In [19]:
checkers

{'Completeness': 'isComplete()',
 'Distinctness': 'isUnique()',
 'Maximum': 'hasMax()',
 'Minimum': 'hasMin()',
 'Size': 'hasSize()'}

In [20]:
modified_attr_dict = {'id':'merchants_id', 'updated_at':'modified_at', 'telephone':'contact_no'}
modified_attr_dict

{'id': 'merchants_id', 'updated_at': 'modified_at', 'telephone': 'contact_no'}

In [21]:
list(zip(instance,values,names))

[('id', 174577.0, 'Maximum'),
 ('telephone', 0.999, 'Completeness'),
 ('id', 1.0, 'Distinctness'),
 ('id', 1.0, 'Completeness'),
 ('id', 469.0, 'Minimum'),
 ('*', 1000.0, 'Size')]

In [84]:
new_columns = filtered.select("instance").rdd.map(lambda r : modified_attr_dict[r[0]] if r[0] in modified_attr_dict.keys() else r[0]).collect()
new_columns

['merchants_id',
 'contact_no',
 'merchants_id',
 'merchants_id',
 'merchants_id',
 '*']

In [35]:
filtered.select("instance","value","name").rdd.map(lambda r : (modified_attr_dict[r[0]],r[1],r[2]) if r[0] in modified_attr_dict.keys() else (r[0],r[1],r[2])).collect()

[('merchants_id', 174577.0, 'Maximum'),
 ('contact_no', 0.999, 'Completeness'),
 ('merchants_id', 1.0, 'Distinctness'),
 ('merchants_id', 1.0, 'Completeness'),
 ('merchants_id', 469.0, 'Minimum'),
 ('*', 1000.0, 'Size')]

In [29]:
filtered.select("instance","value","name").rdd.map(lambda r : r).collect()

[Row(instance='id', value=174577.0, name='Maximum'),
 Row(instance='telephone', value=0.999, name='Completeness'),
 Row(instance='id', value=1.0, name='Distinctness'),
 Row(instance='id', value=1.0, name='Completeness'),
 Row(instance='id', value=469.0, name='Minimum'),
 Row(instance='*', value=1000.0, name='Size')]

In [30]:
filtered.select("instance","value","name").rdd.map(lambda r : r[1]).collect()

[174577.0, 0.999, 1.0, 1.0, 469.0, 1000.0]